In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d ismailpromus/skin-diseases-dataset-processed


100% 2.51G/2.51G [01:51<00:00, 24.0MB/s]
100% 2.51G/2.51G [01:51<00:00, 24.2MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/skin-diseases-dataset-processed.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from tensorflow import keras
from glob import glob
#import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = '/content/Processed TVT/ train'
test_path = '/content/Processed TVT/ test'
valid_path = '/content/Processed TVT/ valid'

In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/Processed TVT/ train/ vasc')
print(folders)
print(len(folders))

NameError: ignored

In [ ]:
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                                   shear_range = 0.2,
                                   rotation_range = 0.4,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   preprocessing_function = preprocess_input)
valid_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 30819 images belonging to 7 classes.


In [ ]:
training_set.class_indices

{'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}

In [ ]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3820 images belonging to 7 classes.


In [ ]:
valid_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3820 images belonging to 7 classes.


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
mc = ModelCheckpoint("./best_model2.h5",
                     monitor = "val_accuracy",
                     verbose = 1, 
                     save_best_only = True)

es = EarlyStopping(monitor = "val_accuracy",
                   min_delta = 0.01,
                   patience = 5,
                   verbose = 1
                   )
cb = [mc,es]

In [ ]:
from tensorflow.keras.optimizers import SGD

In [ ]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=2,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set),
    callbacks = cb
)

<ipython-input-79-48174fe4ea9c>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/2
964/964 [==============================] - ETA: 0s - loss: 4.0574 - accuracy: 0.5500
Epoch 1: val_accuracy improved from -inf to 0.62827, saving model to ./best_model2.h5
964/964 [==============================] - 448s 460ms/step - loss: 4.0574 - accuracy: 0.5500 - val_loss: 3.2767 - val_accuracy: 0.6283
Epoch 2/2
964/964 [==============================] - ETA: 0s - loss: 3.7703 - accuracy: 0.6322
Epoch 2: val_accuracy improved from 0.62827 to 0.65759, saving model to ./best_model2.h5
964/964 [==============================] - 436s 452ms/step - loss: 3.7703 - accuracy: 0.6322 - val_loss: 3.3197 - val_accuracy: 0.6576


In [ ]:
h = r.history
h.keys()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(h['loss'],'go--', c = "red")
plt.plot(h['accuracy'], 'go--')
plt.axis = False
plt.title("Loss vs accuracy")
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

In [ ]:
model = load_model("/content/best_model2.h5")

In [ ]:
path = "/content/Processed TVT/ valid/akiec/ISIC_0024948.jpg"
# path = "/content/Processed TVT/ valid/nv/ISIC_0024309.jpg"
# path = '/content/Processed TVT/ valid/vasc/ISIC_0024370.jpg'
img = load_img(path, target_size = (224, 224))
i = image.img_to_array(img)
i = preprocess_input(i)

input_arr = np.array([i])
# input_arr.shape
pred = np.argmax(model.predict(input_arr))

print("prediction:",pred)

plt.imshow(input_arr[0])